In [4]:
from pyspark.sql import SparkSession

# (8 cores, 16gb per machine) x 5 = 40 cores

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.1.153:7077") \
        .appName("A_LU")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",4)\
        .getOrCreate()
        

# Old API (RDD)
spark_context = spark_session.sparkContext


In [5]:
lines = spark_context.textFile("hdfs://192.168.1.153:9000/europarl/")
print(lines.first())

Състав на Парламента: вж. протоколи


In [6]:
# release the cores for another application!
spark_context.stop()